In [1]:
import numpy as np
import pandas as pd

import ast

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

#movies.head()

In [56]:
#credits.head()

In [59]:
#merging two dataset into one (movies) on column 'title'

movies = movies.merge(credits,on='title')    

In [57]:
#movies.head(1)

In [58]:
#genres
#id
#keywords
#title
#overview
#cast
#crew

movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]    #these are the features which will affect the recommendation process

In [7]:
#movies['original_language'].value_counts()

movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [8]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   overview  4806 non-null   object
 3   genres    4809 non-null   object
 4   keywords  4809 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 263.1+ KB


In [10]:
movies = movies.dropna()      #to remove null values

movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [11]:
movies.duplicated().sum()

0

In [12]:
movies.iloc[0].genres      #first row

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [60]:
#to convert dictionary format to list 

def convert(obj):         
    L = []
    for i in ast.literal_eval(obj):      #string to list conversion - ast.literal_eval()
        L.append(i['name'])
    return L

In [61]:
#ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [15]:
movies['genres'] = movies['genres'].apply(convert)

In [16]:
movies['keywords'] = movies['keywords'].apply(convert)

In [17]:
#movies.head(1)

In [18]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:               #only top 3 actors/casts
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [19]:
movies['cast'] = movies['cast'].apply(convert3)

In [20]:
#movies.head(1)

In [21]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':      #only director
            L.append(i['name'])
            break
    return L

In [22]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [23]:
#movies.head(1)

In [24]:
#string to list conversion

movies['overview'] = movies['overview'].apply(lambda x:x.split())     

In [25]:
#to remove spaces between two or more words (for each item in list)

movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])        
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [26]:
#movies.head(1)

In [27]:
 #concatenate all the lists into one

movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']     

In [28]:
#movies.head(1)

In [29]:
new_df = movies[['movie_id','title','tags']]

#new_df.head()

In [30]:
#list to string conversion

new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))     #words are joined by space (each list)

C:\Users\USER\AppData\Local\Temp\ipykernel_22344\3141862357.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [31]:
#new_df.head()

In [32]:
#lower case conversion

new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\USER\AppData\Local\Temp\ipykernel_22344\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [33]:
#new_df.head() 

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


In [34]:
#vectorization 

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [35]:
#vectors = cv.fit_transform(new_df['tags']).toarray()

In [36]:
#vector.shape

In [37]:
#!pip install nltk

In [38]:
import nltk

In [39]:
#stemming

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [40]:
def stem (text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [41]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\USER\AppData\Local\Temp\ipykernel_22344\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [42]:
#new_df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."


In [43]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [44]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [45]:
#vectors

In [46]:
#vectors.shape

In [47]:
#cosine distance

from sklearn.metrics.pairwise import cosine_similarity

In [48]:
similarity = cosine_similarity(vectors)

In [52]:
#similarity 

In [54]:
def recommend(movie):
    movie_index = new_df[new_df ['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key = lambda x: x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title) 

In [55]:
recommend('Gandhi')

Gandhi, My Father
Guiana 1838
The Wind That Shakes the Barley
Mr. Turner
The Sea Inside
